# Initializing the Herbie object and downloading the file subsets

In [ ]:
from herbie import Herbie
import xarray as xr
import pandas as pd

start_time = pd.to_datetime('2025-04-11 12:00')
v_list = []
other_list = []
mu_list = []
for fxx in range(0,49):
    H = Herbie(start_time, model='hrrr', product='prs', priority='google', fxx=fxx)
    path_v = H.download(":VUCSH:0-6000 m above ground:|:VVCSH:0-6000 m above ground:")
    path_other = H.download(":MXUPHL:5000-2000 m above ground:|:REFC:entire atmosphere:|:CIN:surface:|:CAPE:surface|:DPT:surface")
    path_muc = H.download("CAPE:180-0 mb above ground")
    ds_mu = xr.open_dataset(path_muc, engine='cfgrib')
    ds_v = xr.open_dataset(path_v, engine='cfgrib')
    ds_other = xr.open_dataset(path_other, engine='cfgrib')
    v_list.append(ds_v)
    other_list.append(ds_other)
    mu_list.append(ds_mu)
combined_v = xr.concat(v_list, dim="step")
combined_other = xr.concat(other_list, dim="step")
combined_mu = xr.concat(mu_list, dim='step')
print(combined_other)


# Loading the variables and accessing the data underlying data arrays

In [ ]:
conv = combined_other
shr = combined_v
mu = combined_mu
uphl = conv.unknown.values
ref = conv.refc.values
sbcin = conv.cin.values
v = shr.vvcsh.values
u = shr.vucsh.values
lat = conv.latitude.values
lon = conv.longitude.values
v_time = conv.valid_time.values[0]
formatted_time = pd.to_datetime(v_time).strftime('%Y-%m-%d %H:%M')
valid_times = conv.valid_time.values
run_time = conv.valid_time.values[0]
hour_int = pd.to_datetime(run_time).hour
# sfc_cp = conv.cape.values
# mu_cp = mu.cape.values

# Plotting

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import numpy as np
from metpy.plots import USCOUNTIES
from scipy.ndimage import gaussian_filter
import imageio  
from datetime import datetime, timedelta

# Assume formatted_time is defined as a string (e.g., "2025-03-05 12:00")
# Create a datetime object for the run time
run_time = datetime.strptime(formatted_time, '%Y-%m-%d %H:%M')

# Define a center and a "radius" to each side
central_lat = 41.8781
central_lon = -87.6298
extent = 4.6
north = central_lat + extent
south = central_lat - extent
east  = central_lon + extent
west  = central_lon - extent

# Coordinates for Chicago, IL 
chicago_lat = 41.8781
chicago_lon = -87.6298

# We'll store the filenames we generate
filenames = []

# Loop through the frames (each frame is +1 hour from run_time)
for t in range(0, 49):
    fig, ax = plt.subplots(
        figsize=(10, 8),
        subplot_kw={'projection': ccrs.PlateCarree()},
        dpi=150
    )
    
    # Set the extent using our computed bounding box
    ax.set_extent([west, east, south, north], crs=ccrs.PlateCarree())

    # Add geographic features
    ax.add_feature(cfeature.LAND, facecolor='white')
    ax.add_feature(cfeature.COASTLINE, linewidth=1.2)
    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=1.2, linestyle=':')
    ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=1.2)
    ax.add_feature(USCOUNTIES.with_scale('20m'), edgecolor='gray', linewidth=0.5)

    # --- Plot a point for Greenville, MS (marker only, no annotation) ---
    ax.plot(chicago_lon, chicago_lat, marker='o', color='red', markersize=8,
             transform=ccrs.PlateCarree())

    # --- Plot reflectivity ---
    ref2d = ref[t] if ref.ndim == 3 else ref
    norm = mcolors.Normalize(vmin=0, vmax=80)
    levels_ref = np.linspace(0, 80, int(80 / 2) + 1)
    cf_ref = ax.contourf(
        lon, lat, ref2d,
        levels=levels_ref,
        cmap="Pivotal_REF",    # or e.g., "Radarscope_BR"
        norm=norm,
        alpha=1,
        transform=ccrs.PlateCarree(),
        extend='max', zorder=2
    )

    # --- Updraft Helicity (UH) > 75 ---
    uphl2d = uphl[t] if uphl.ndim == 3 else uphl
    uh_smooth = gaussian_filter(uphl2d, sigma=0.5)
    # Filled contour for UH above 75
    ax.contourf(
        lon, lat, uh_smooth,
        levels=[75, 9999],
        colors='#3D1C02',
        alpha=0.4,
        transform=ccrs.PlateCarree(),zorder=2
    )
    # Outline contour at exactly 75
    ax.contour(
        lon, lat, uh_smooth,
        levels=[75],
        colors='#3D1C02',
        linewidths=1.2,
        alpha=1.0,
        transform=ccrs.PlateCarree(),zorder=2
    )

    ratio = sfc_cp[t] if sfc_cp.ndim == 3 else sfc_cp / mu_cp [t] if mu_cp[t] != 0 else np.nan

    
    ratio = gaussian_filter(ratio, sigma=1)
    # ratio2d = ratio[t] if ratio.ndim == 3 else ratio 


    # rt = ax.contourf(lon, lat, ratio,
    #             levels=np.arange(0, 1.51, 0.1),
    #             cmap='bwr',
    #             alpha=0.4,
    #             transform=ccrs.PlateCarree(), zorder=1
    #            )

    

    # --- CIN < -100 (hatching) ---
    sbcin2d = sbcin[t] if sbcin.ndim == 3 else sbcin
    cin_smooth = gaussian_filter(sbcin2d, sigma=1)
    cin_mask = np.where(cin_smooth < -100, 1, np.nan).astype(float)
    ax.contourf(
        lon, lat, cin_mask,
        levels=[0.5, 1.5],
        colors="none",
        hatches=["//"],
        transform=ccrs.PlateCarree(),zorder=2
    )
    hatch_patch = mpatches.Patch(
        facecolor='none', hatch='//', edgecolor='black',
        label='SB CIN < -100 J kg$^{-1}$'
    )
    legend = ax.legend(handles=[hatch_patch], loc='upper right', fontsize=9)
    legend.get_frame().set_linewidth(0.8)

    # --- Wind Barbs ---
    barb_step = 20
    u2d = u[t] if u.ndim == 3 else u
    v2d = v[t] if v.ndim == 3 else v
    lon_barbs = lon[::barb_step, ::barb_step]
    lat_barbs = lat[::barb_step, ::barb_step]
    u_barbs   = u2d[::barb_step, ::barb_step]
    v_barbs   = v2d[::barb_step, ::barb_step]
    ax.barbs(
        lon_barbs, lat_barbs,
        u_barbs, v_barbs,
        transform=ccrs.PlateCarree(),
        length=6, alpha=0.9,
        sizes=dict(spacing=0.2, height=0.5, emptybarb=0.1),
        linewidth=0.8,zorder=2
    )

    # --- Time Annotations ---
    # Compute the valid time for this frame (each frame is +1 hour)
    valid_time = run_time + timedelta(hours=t)
    # Title on the left displays the constant Run Time and additional info
    ax.set_title(
        f"HRRR {run_time.strftime('%m/%d/%Y %HZ')}\n0-6 km shear (barbs), Composite Reflectivity, Updraft Helicity ≥ 75 $m^2 s^2$",
        fontsize=8,
        loc='left',
        fontweight='bold'
    )
    # Add a separate text for the updating Valid Time
    ax.set_title(
        f"Valid Time: {valid_time.strftime('%m/%d/%Y %HZ')}",
        fontsize=8,
        loc='right',
        fontweight='bold',
        color='black'
    )

    # --- Colorbar ---
    cax_ref = fig.add_axes([0.88, 0.15, 0.02, 0.7])
    cbar = fig.colorbar(cf_ref, cax=cax_ref, orientation="vertical", label="Composite Reflectivity (dBZ)")
    cbar.set_ticks(np.arange(0, 80, 5))
    # cbar_r = plt.colorbar(
    #     rt, ax=ax,
    #     orientation = 'horizontal',
    #     label = '← Elevated       Surface Based →',
    #     fraction =0.046, pad=0.02, shrink=0.96
    # )

    # cbar.set_ticks(np.arange(0, 80, 5))

    plt.tight_layout()

    # Save the frame
    outname = f"frame_{t:02d}.png"
    plt.savefig(outname, dpi=300, bbox_inches='tight')
    plt.close(fig)
    filenames.append(outname)

# Build the GIF from saved frames
images = []
for fname in filenames:
    images.append(imageio.imread(fname))
imageio.mimsave('my_animation_r.gif', images, loop=0, fps=4)
print("Saved GIF => my_animation.gif")
